In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from os import listdir
from os.path import isfile, join

import seaborn as sns
from progressbar import ProgressBar
from sklearn.preprocessing import MinMaxScaler
import scipy
from tqdm import tqdm

import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from tensorflow.keras.layers import Dense, Input, Conv1D, MaxPooling1D, Reshape, Flatten, GlobalMaxPooling1D, Layer
from tensorflow.keras.layers import Dropout, ReLU
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import MeanSquaredError
from keras.initializers import glorot_normal
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

from tensorflow.keras import Model
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as rmse

plt.rcParams.update({'font.size': 18})

2022-12-19 11:39:27.975252: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-19 11:39:28.932743: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-19 11:39:28.932826: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-19 11:39:28.932834: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

## Load the new CMAPPS data

In [3]:
mypath = 'N-CMAPSS_DL/N-CMAPSS/Samples_whole/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)

train_units = [2,5,10,16,18,20]
test_units = [11,14,15]

test_files = [[f for f in onlyfiles if str(test_units[j]) in f ] for j in range(len(test_units))]
test_files = np.array(test_files)[:,0].tolist()
train_files = list(set(onlyfiles) - set(test_files))

['Unit11_win1_str1_smp10.npz', 'Unit2_win1_str1_smp10.npz', 'Unit10_win1_str1_smp10.npz', 'Unit14_win1_str1_smp10.npz', 'Unit5_win1_str1_smp10.npz', 'Unit18_win1_str1_smp10.npz', 'Unit16_win1_str1_smp10.npz', 'Unit15_win1_str1_smp10.npz', 'Unit20_win1_str1_smp10.npz']


In [4]:
def load_part_array_merge (current_dir, npz_units):
    sample_array_lst = []
    label_array_lst = []
    for npz_unit in npz_units:
      loaded = np.load(current_dir + npz_unit)
      sample_array_lst.append(loaded['sample'])
      label_array_lst.append(loaded['label'])
    sample_array = np.dstack(sample_array_lst)
    label_array = np.concatenate(label_array_lst)
    sample_array = sample_array.transpose(2, 0, 1)
    return sample_array, label_array

## Make Input-Output for training the UQ models

In [5]:
Xtrain_norm,Ytrain = load_part_array_merge(mypath,train_files)
Xtest_norm,Ytest = load_part_array_merge(mypath,test_files)
Xtrain_norm = Xtrain_norm[:,0,:]
Xtest_norm = Xtest_norm[:,0,:]
Ytrain_norm = (Ytrain)/np.max(Ytrain)
Ytest_norm = (Ytest)/np.max(Ytrain)

## Uncertainty Model 1: Deep Ensemble (DE)

- Use of Gaussian Layer that outputs - mean and aleatoric variance
- Use of Negative Log Likelihood loss
- Ensemble to capture epistematic uncertainty

In [7]:
# loss function and new custom layer to give mean and standard deviation
def custom_loss(sigma):
    def gaussian_loss(y_true, y_pred):
        return tf.reduce_mean(0.5*tf.math.log((sigma)) + 0.5*tf.math.divide(tf.math.square(y_true - y_pred), (sigma))) + 1e-6
    return gaussian_loss
class GaussianLayer(Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(GaussianLayer, self).__init__(**kwargs)
    def build(self, input_shape):
        self.kernel_1 = self.add_weight(name='kernel_1', 
                                      shape=(10, self.output_dim),
                                      initializer=glorot_normal(),
                                      trainable=True)
        self.kernel_2 = self.add_weight(name='kernel_2', 
                                      shape=(10, self.output_dim),
                                      initializer=glorot_normal(),
                                      trainable=True)
        self.bias_1 = self.add_weight(name='bias_1',
                                    shape=(self.output_dim, ),
                                    initializer=glorot_normal(),
                                    trainable=True)
        self.bias_2 = self.add_weight(name='bias_2',
                                    shape=(self.output_dim, ),
                                    initializer=glorot_normal(),
                                    trainable=True)
        super(GaussianLayer, self).build(input_shape) 
    def call(self, x):
        output_mu  = K.dot(x, self.kernel_1) + self.bias_1
        output_sig = K.dot(x, self.kernel_2) + self.bias_2
        output_sig_pos = K.log(1 + K.exp(output_sig)) + 1e-06  
        return [output_mu, output_sig_pos]
    def compute_output_shape(self, input_shape):
        return [(input_shape[0], self.output_dim), (input_shape[0], self.output_dim)]

In [8]:
def generate_trained_DEmodel(trainX, trainY, nepochs, actfn = 'sigmoid'):
    n_inp_features = np.shape(trainX)[1]
    feature_input = Input(shape=(n_inp_features,), name="feature_input_layer")
    x = Dense(100, activation = actfn)(feature_input)
    x = Dense(10, activation = actfn)(x)
      
    mu, sigma = GaussianLayer(1, name='main_output')(x)
    model = Model(feature_input, mu)
    model.compile(loss=custom_loss(sigma), optimizer='adam')
    model.fit(trainX, trainY,shuffle=True, epochs=nepochs, verbose = 1)
    
    layer_name = 'main_output' # Where to extract the Gaussian output from. 
    get_intermediate = K.function(inputs=[model.input], outputs=model.get_layer(layer_name).output)
    return get_intermediate

In [ ]:
## Create 20 models and select some based on training error
prediction_fns, train_mae = [], []
nmodels = 2
nepochs = 20

pbar=ProgressBar()
for i in pbar(range(nmodels)):
    my_func = generate_trained_DEmodel(Xtrain_norm, Ytrain_norm, nepochs, actfn = 'tanh')
    train_mae.append(mae(Ytrain, (my_func(Xtrain_norm)[0])*np.max(Ytrain)))
    prediction_fns.append(my_func)


Train on 526345 samples
Epoch 1/20


2022-12-19 11:39:58.237903: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-12-19 11:39:58.238609: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-12-19 11:39:58.238630: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-12-19 11:39:58.239643: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI

526345/526345 [==============================] - 24s 46us/sample - loss: -1.7489
Epoch 2/20
526345/526345 [==============================] - 24s 45us/sample - loss: -2.1300
Epoch 3/20
526345/526345 [==============================] - 24s 45us/sample - loss: -2.2111
Epoch 4/20
526345/526345 [==============================] - 24s 45us/sample - loss: -2.2580
Epoch 5/20
526345/526345 [==============================] - 24s 45us/sample - loss: -2.2871
Epoch 6/20
526345/526345 [==============================] - 24s 45us/sample - loss: -2.3079
Epoch 7/20
526345/526345 [==============================] - 24s 45us/sample - loss: -2.3311
Epoch 8/20
526345/526345 [==============================] - 24s 45us/sample - loss: -2.3548
Epoch 9/20
526345/526345 [==============================] - 24s 46us/sample - loss: -2.3695
Epoch 10/20
526345/526345 [==============================] - 24s 45us/sample - loss: -2.3792
Epoch 11/20
526345/526345 [==============================] - 24s 46us/sample - loss: -2.38

In [ ]:
nmodels_select = 2
final_DE_fns = []
for i in range(nmodels_select):
    final_DE_fns.append(prediction_fns[np.argsort(train_mae)[:nmodels_select][i]])

In [ ]:
def get_ensemble(df, UQmodelcode, nmodels_select):
    mu_cols = [UQmodelcode + "_mu_"+str(i) for i in range(nmodels_select)]
    sigma_cols = [UQmodelcode + "_sigma_"+str(i) for i in range(nmodels_select)]
    
    mu_eff = df[mu_cols].mean(axis=1)
    if UQmodelcode=="DE":
        sigma_eff =  np.sqrt(np.mean(df[mu_cols]**2, axis = 1)
                             + np.mean(df[sigma_cols]**2, axis = 1)
                             - mu_eff**2)
    elif UQmodelcode=="MC":
        sigma_eff =  np.sqrt(np.mean(df[mu_cols]**2, axis = 1)
                             - mu_eff**2)
    df[UQmodelcode+"_mu_eff"] = mu_eff
    df[UQmodelcode+"_sigma_eff"] = sigma_eff
    return df

allresults_df = pd.DataFrame()
dataset = ['train', 'test']
for myset in dataset:
    exec("Xnorm =X"+myset+"_norm")
    exec("Y =Y"+myset)

    result_df = pd.DataFrame(Y, columns = ['True'])
    result_df['dataset'] = myset
    for i in range(nmodels_select):
        result_df["DE_mu_"+str(i)] = (final_DE_fns[i](Xnorm)[0])*np.max(Ytrain)
        result_df["DE_sigma_"+str(i)] = (np.sqrt(final_DE_fns[i](Xnorm)[1]))*np.max(Ytrain)

    result_df = get_ensemble(result_df, "DE", nmodels_select)
    allresults_df = pd.concat([allresults_df, result_df], axis = 0)
allresults_df.head()

## Results - Visualization

In [ ]:
y_true = allresults_df[allresults_df['dataset']=='test']['True'].values
y_pred = allresults_df[allresults_df['dataset']=='test']['DE_mu_0'].values
sigma_pred = allresults_df[allresults_df['dataset']=='test']['DE_sigma_0'].values
plt.figure()
plt.plot(y_pred)
plt.plot(y_true)
plt.figure()
plt.plot(sigma_pred)
plt.ylim(0,50)

In [ ]:
def get_confidence_interval(y_true, mu, sigma):
    alphas = np.linspace(1e-10, 1-1e-10, 1000)
    myCI=[]
    for myalpha in tqdm(sorted(alphas)):
        intervals = scipy.stats.norm.interval(alpha=myalpha, loc=mu, scale=sigma)
        lower_bd = intervals[0]
        upper_bd = intervals[1]
        myCI.append(np.sum((y_true > lower_bd) & (y_true < upper_bd))/len(y_true))
    
    return 100*alphas, 100*np.array(myCI)  # converting to percentages

In [ ]:
alphas, CI_DE = get_confidence_interval(allresults_df_sorted['True'].values, 
                                       allresults_df_sorted['DE_mu_eff'].values, 
                                       allresults_df_sorted['DE_sigma_eff'].values)


In [ ]:
fig = plt.figure(figsize=(8,6))
plt.plot(alphas,CI_DE, color = 'blue', label='Deep Ensemble')
plt.plot([0,100],[0,100], color='black', linestyle='dashed', label='Ideal')
plt.xlabel('Expected Confidence')
plt.ylabel('Precited Confidence')
plt.legend()

## Repeat the above algorithm for 10 times to capture variations

In [ ]:
## Multiple runs
niter = 10
allCI_DE = np.zeros((1000, niter))
pbar=ProgressBar()

for myiter in pbar(range(niter)):
    prediction_fns=[]
    train_mae=[]
    for i in range(nmodels):
        my_func = generate_trained_DEmodel(Xtrain_norm, Ytrain_norm, nepochs, actfn = 'tanh')
        train_mae.append(mae(Ytrain, (my_func(Xtrain_norm)[0])*np.max(Ytrain)))
        prediction_fns.append(my_func)

    nmodels_select = 2
    final_DE_fns = []
    for i in range(nmodels_select):
        final_DE_fns.append(prediction_fns[np.argsort(train_mae)[:nmodels_select][i]])

    allresults_df = pd.DataFrame()
    for myset in dataset:
        exec("Xnorm =X"+myset+"_norm")
        exec("Y =Y"+myset)

        result_df = pd.DataFrame(Y, columns = ['True'])
        result_df['dataset'] = myset
        for i in range(nmodels_select):
            result_df["DE_mu_"+str(i)] = (final_DE_fns[i](Xnorm)[0])*np.max(Ytrain)
            result_df["DE_sigma_"+str(i)] = (np.sqrt(final_DE_fns[i](Xnorm)[1]))*np.max(Ytrain)

        result_df = get_ensemble(result_df, "DE", nmodels_select)
        allresults_df = pd.concat([allresults_df, result_df], axis = 0)
    allresults_df.head()

    allresults_df_sorted = allresults_df.sort_values(by='True', ascending=True)

    alphas, CI_DE = get_confidence_interval(allresults_df_sorted['True'].values, 
                                           allresults_df_sorted['DE_mu_eff'].values, 
                                           allresults_df_sorted['DE_sigma_eff'].values)
    allCI_DE[:,myiter] = CI_DE

In [ ]:
fig = plt.figure(figsize=(8,6))
plt.plot([0,100],[0,100], color='black', linestyle='dashed', label='Ideal')
plt.fill_between(alphas,
                np.mean(allCI_DE, axis = 1)+np.std(allCI_DE, axis = 1),
                np.mean(allCI_DE, axis = 1)-np.std(allCI_DE, axis = 1),
                color='blue', alpha = 0.4, linestyle='None', label='Deep Ensemble')
plt.xlabel('Expected Confidence')
plt.ylabel('Precited Confidence')
plt.legend()

## UQ Model: MC Dropout

Simple MLP architecture with 10% dropout probability

In [ ]:
def generate_trained_MCmodel(trainX, trainY, nepochs, actfn = 'sigmoid'):

    n_inp_features = np.shape(trainX)[1]
    feature_input = Input(shape=(n_inp_features,), name="feature_input_layer")
    x = Dense(100, activation = actfn)(feature_input)
    x = Dropout(rate = 0.10)(x)
    x = Dense(10, activation = actfn)(x)
    x = Dropout(rate = 0.10)(x)
    mu = Dense(1, activation = actfn)(x)

    model = Model(feature_input, mu)
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(trainX, trainY,shuffle=True, epochs=nepochs, verbose = 1)

    return model

In [ ]:
nmodels = 2
all_models_MC=[]
all_mae_train_MC = np.zeros(nmodels,)
pbar = ProgressBar()
for i in pbar(range(nmodels)):
    model_MC = generate_trained_MCmodel(Xtrain_norm, Ytrain_norm, 20, 'sigmoid')
    all_mae_train_MC[i] = mae(K.get_value(model_MC(Xtrain_norm))*(np.max(Ytrain)), Ytrain)
    all_models_MC.append(model_MC)

In [ ]:
select_modelMC = all_models_MC[np.argmin(all_mae_train_MC)]
print("Train MAE: ", mae(K.get_value(select_modelMC(Xtrain_norm))*(np.max(Ytrain)), Ytrain))
print("Test1 MAE: ", mae(K.get_value(select_modelMC(Xtest_norm))*(np.max(Ytrain)), Ytest))


In [ ]:
allresults_MCdf = pd.DataFrame()
for myset in dataset:
    exec("Xnorm =X"+myset+"_norm")
    exec("Y =Y"+myset)

    result_df = pd.DataFrame(Y, columns = ['True'])
    result_df['dataset'] = myset
    for i in range(nmodels_select):
        result_df["MC_mu_"+str(i)] = K.get_value(select_modelMC(Xnorm, training=True))*(np.max(Ytrain))

    allresults_MCdf = pd.concat([allresults_MCdf, result_df], axis = 0)
allresults_MCdf = get_ensemble(allresults_MCdf, "MC", nmodels_select)

## Results - Visualization

In [ ]:
alphas, CI_DE = get_confidence_interval(allresults_MCdf['True'].values, 
                                       allresults_MCdf['MC_mu_eff'].values, 
                                       allresults_MCdf['MC_sigma_eff'].values)
fig = plt.figure(figsize=(8,6))
plt.plot(alphas,CI_DE, color = 'blue', label='MC Dropout')
plt.plot([0,100],[0,100], color='black', linestyle='dashed', label='Ideal')
plt.xlabel('Expected Confidence')
plt.ylabel('Precited Confidence')
plt.legend()

## Repeat the above algorithm for 10 times to capture variations

In [ ]:
## Repeat 10 times
nmodels = 5 # total models trained and best one picked
nmodels_select = 5 # run 5 times per each select model with dropout
niter = 10

allCI_MC = np.zeros((1000, niter))
pbar=ProgressBar()

for myiter in pbar(range(niter)):
    all_models_MC=[]
    all_mae_train_MC = np.zeros(nmodels,)
    pbar = ProgressBar()
    for i in range(nmodels):
        model_MC = generate_trained_MCmodel(Xtrain_norm, Ytrain_norm, 3000, 'sigmoid')
        all_mae_train_MC[i] = mae(K.get_value(model_MC(Xtrain_norm))*(np.max(Ytrain)), Ytrain)
        all_models_MC.append(model_MC)

    select_modelMC = all_models_MC[np.argmin(all_mae_train_MC)]

    allresults_MCdf = pd.DataFrame()
    for myset in dataset:
        exec("Xnorm =X"+myset+"_norm")
        exec("Y =Y"+myset)

        result_df = pd.DataFrame(Y, columns = ['True'])
        result_df['dataset'] = myset
        for i in range(nmodels_select):
            result_df["MC_mu_"+str(i)] = K.get_value(select_modelMC(Xnorm, training=True))*(np.max(Ytrain))

        allresults_MCdf = pd.concat([allresults_MCdf, result_df], axis = 0)
    allresults_MCdf = get_ensemble(allresults_MCdf, "MC", nmodels_select)

    alphas, CI_MC = get_confidence_interval(allresults_MCdf['True'].values, 
                                           allresults_MCdf['MC_mu_eff'].values, 
                                           allresults_MCdf['MC_sigma_eff'].values)
    allCI_MC[:,myiter] = CI_MC

In [ ]:
fig = plt.figure(figsize=(8,6))
plt.plot([0,100],[0,100], color='black', linestyle='dashed', label='Ideal')
plt.fill_between(alphas,
                np.mean(allCI_MC, axis = 1)+np.std(allCI_MC, axis = 1),
                np.mean(allCI_MC, axis = 1)-np.std(allCI_MC, axis = 1),
                color='red', alpha = 0.4, linestyle='None', label='MC-Dropout')
plt.fill_between(alphas,
                np.mean(allCI_DE, axis = 1)+np.std(allCI_DE, axis = 1),
                np.mean(allCI_DE, axis = 1)-np.std(allCI_DE, axis = 1),
                color='blue', alpha = 0.4, linestyle='None', label='Deep Ensemble')

plt.xlabel('Expected Confidence')
plt.ylabel('Observed Confidence')
plt.legend()